# MENA Analysis: Air Quality PM2.5

### Load packages and setup

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterstats import zonal_stats
import rasterio

In [3]:
# Define directories
dir_shp = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Boundaries/'
dir_in = '/Users/ashitakarl/Desktop/Project/WB/MENA/SEDAC/'
dir_out = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Index/ADM2/'

In [3]:
%matplotlib inline

### Load shapefile

In [4]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')

MENA_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Load PM2.5 rasters

In [5]:
#from shapely.geometry import mapping
#import rioxarray as rxr
#from rasterio.plot import show
#pm_2000 = rasterio.open(dir_in + '2000/sdei-global-annual-gwr-pm2-5-modis-misr-seawifs-aod-v4-gl-03-2000.tif')
#pm_2000 = rxr.open_rasterio(dir_in + '2000/sdei-global-annual-gwr-pm2-5-modis-misr-seawifs-aod-v4-gl-03-2000.tif',
#                            masked=True).squeeze()
# Clip the raster to the extent of polygon
#pm_2000_clip = pm_2000.rio.clip(MENA_shp.geometry.apply(mapping),
#                                MENA_shp.crs)

In [5]:
pm_full = pd.DataFrame(MENA_shp['ID_ADM'])

for YEAR in [2000, 2005, 2010, 2015, 2019]:
    with rasterio.open(dir_in + str(YEAR) + 
                       '/sdei-global-annual-gwr-pm2-5-modis-misr-seawifs-aod-v4-gl-03-' + 
                       str(YEAR) + '.tif') as pm:
        array = pm.read(1)
        affine = pm.transform
        nodata = pm.nodata

    MENA_shp.to_crs(pm.crs, inplace = True)

    # Extract raster by polygon
    pm_dict = zonal_stats(MENA_shp, array, affine = affine, nodata = nodata,
                          interpolate = 'bilinear', all_touched = True,
                          stats = ['min', 'mean', 'max', 'median', 'std', 'count'])
    pm_df = pd.DataFrame(pm_dict)
    pm_df = pm_df.add_suffix('_' + str(YEAR))
    pm_df = pm_df.add_prefix('pm25_')
    pm_df.insert(0, 'ID_ADM', MENA_shp['ID_ADM'])
    
    pm_full = pm_full.merge(pm_df, how = 'left', on = 'ID_ADM')

In [7]:
pm_full.pm25_mean_2000.isna().sum().sum()

19

In [13]:
pm_full.pm25_median_2000.mean()

23.381065823265427

In [6]:
pm_full

,ID_ADM,pm25_min_2000,pm25_max_2000,pm25_mean_2000,pm25_count_2000,pm25_std_2000,pm25_median_2000,pm25_min_2005,pm25_max_2005,pm25_mean_2005,...,pm25_mean_2015,pm25_count_2015,pm25_std_2015,pm25_median_2015,pm25_min_2019,pm25_max_2019,pm25_mean_2019,pm25_count_2019,pm25_std_2019,pm25_median_2019
0,37631,59.900002,65.800003,62.277732,705,0.839842,62.299999,57.099998,65.500000,60.450355,...,56.050920,705,1.626632,55.700001,36.900002,43.099998,38.857164,705,1.051191,38.599998
1,37632,68.800003,72.599998,71.157635,1664,0.796410,71.300003,62.900002,68.199997,65.715989,...,62.970975,1664,1.170739,63.099998,45.500000,52.200001,48.746817,1664,1.270269,48.900002
2,37633,46.900002,62.099998,54.395429,13434,2.724675,54.400002,46.799999,59.599998,52.701536,...,47.848095,13434,1.314940,48.000000,30.100000,36.299999,32.979022,13434,1.060964,33.000000
3,37634,66.099998,71.500000,68.491216,2928,1.264061,68.199997,60.099998,68.000000,64.653486,...,61.384189,2928,2.013333,61.599998,39.700001,49.599998,43.856659,2928,2.069195,43.500000
4,37635,60.400002,74.000000,67.315558,107286,2.366561,67.500000,58.000000,70.699997,63.665525,...,66.000443,107286,3.697681,65.800003,40.200001,69.500000,58.402639,107286,6.334601,60.400002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,15561,22.100000,29.299999,24.659613,3605,1.202665,24.600000,21.500000,24.700001,22.809514,...,23.386382,3605,0.862439,23.200001,20.600000,25.200001,22.752761,3605,0.996658,22.799999
2933,15562,20.799999,34.799999,29.220845,147760,2.343809,29.700001,26.100000,44.099998,33.577216,...,29.966554,147760,3.826548,29.400000,24.600000,48.400002,35.713383,147760,4.842750,35.400002
2934,15563,26.500000,34.700001,28.495722,3740,1.011222,28.400000,26.900000,39.599998,31.324653,...,34.998423,3740,1.548479,34.799999,33.000000,47.799999,39.528956,3740,2.112242,39.599998
2935,15564,17.000000,31.100000,23.165536,25866,2.934806,23.200001,18.299999,27.400000,23.329453,...,25.327447,25866,1.345892,25.200001,19.900000,27.400000,23.583698,25866,1.317999,23.500000


In [14]:
pm_full.to_csv(dir_out + 'ADM2_Air_Quality_2000_2019.csv', index = False)